# Эволюция клеточного автомата "Муравей Лэнгтона" на торических решетках

## Краткое описание

Муравей Лэнгтона — это двумерный клеточный автомат с очень простыми правилами, изобретенный Крисом Лэнгтоном. Муравья можно также считать двумерной машиной Тьюринга с 2 символами и 4 состояниями.

## Правила движения

Рассмотрим бесконечную плоскость, разбитую на клетки, покрашенные некоторым образом в чёрный и белый цвет. Пусть в одной из клеток находится «муравей», который на каждом шаге может двигаться в одном из четырёх направлений в клетку, соседнюю по стороне. Муравей движется согласно следующим правилам:

1. На чёрном квадрате — повернуть на 90° влево, изменить цвет квадрата на белый, сделать шаг вперед на следующую клетку.
2. На белом квадрате — повернуть на 90° вправо, изменить цвет квадрата на чёрный, сделать шаг вперед на следующую клетку.

![AntUrl](https://upload.wikimedia.org/wikipedia/commons/0/09/LangtonsAntAnimated.gif "Ant")

## Ключевые особенности реализуемого алгоритма

Опишем интересующую нас версию муравья Лэнгтона, формулируем и докажем несколько гипотез о его эволюции. Как известно, тор – это сплошная фигура, образованная вращением круга вокруг прямой, лежащей в плоскости этого круга, но не пересекающей его. 

В нашем случае муравей будет перемещаться по плоскому тору (торической решётке). Плоский тор – это фигура, получающаяся из квадрата склейкой его противоположных сторон. Если представить себе квадрат и соединить его верхнюю границу с нижней, мы получим фигуру, похожую на цилиндр. Если затем соединить края цилиндра друг с другом, то получится тор. Мы будем предполагать, что наше поле – это решётка на торе, полученная в результате такой операции и использовать для неё термин «торическая решётка».

Понятно, что основное отличие от случая эволюции на плоскости заключается в том, что теперь поле состоит из конечного числа клеток. Поскольку количество всевозможных раскрасок этих клеток в два цвета и количество возможных положений муравья конечно, рано или поздно возникнет конфигурация, которая уже была. Более того, понятно, что по данной конфигурации мы можем восстановить предыдущую однозначно, откуда следует, что первая повторившаяся конфигурация – это и есть самая первая конфигурация.

Мы будем рассматривать случай, когда изначально все клетки торической решётки белые. Таким образом, основной вопрос заключается в том, через какое число ходов мы получим снова белое поле с первоначальным положением муравья. Также интересно, когда поле станет белым впервые (независимо от того, в какой клетке находится и куда смотрит муравей).


## Решение задачи

Импортируем необходимые библиотеки

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import moviepy.editor as mpy
import pandas as pd

Введем размер торической решетки, ее цвет и нальное положение и напрвление взляда муравья муравья

In [2]:
print('Введите размер матрицы: ')
matrix_size = str('4 4').split()
matrix_size = tuple(map(int, matrix_size))
print(matrix_size)

Введите размер матрицы: 
(4, 4)


In [3]:
print('Введите цвет матрицы: ')
start_color = str('white').lower()
print(start_color)

Введите цвет матрицы: 
white


In [4]:
print('Введите начальное положение муравья: ')
start_location = str('0 0').split()
start_location = tuple(map(int, start_location))
print(start_location)

Введите начальное положение муравья: 
(0, 0)


In [5]:
print('Введите направление муравья: ')
start_direction = str('up').lower()
directions = {'up': '^', 'down': 'v', 'left': '<', 'right': '>'}
print(start_direction)

Введите направление муравья: 
up


Укажем директорию для сохранения промежуточных результатов

In [6]:
# save_directory = 'C:/'

Создадим двумерные массивы, отражающие текущее значение цвета и положения муравья

In [7]:
colors_dict = {'black': 0, 'white': 1} 
color = colors_dict.get(start_color)

# Матрица цвета
matrix_color = [[color for columns in range(matrix_size[0])]
                for rows in range(matrix_size[1])]

# Переменная, исмользуемая для проверки возвращения цвета в исходное состояние
check_color = 0 

In [8]:
# Матрица положения муравья
matrix_location = [[int(0) for column in range(matrix_size[0])] for row in range(matrix_size[1])]

# Указание начального положения муравья в матрице
matrix_location[start_location[1]][start_location[0]] = int(1)

# Переменная, отражающая текущее положение в матрице
current_location = list(start_location)

# Переменная, отражающая текущее направление взгляда муравья
current_direction = start_direction

Опишем функции, отвечающие за движение муравья в каждом направлении

Движение влево:

In [9]:
def go_left():
    global matrix_color, matrix_location, current_location, current_direction, check_color
    # Изменение цвета ячейки
    if matrix_color[current_location[1]][current_location[0]] == 0:
        matrix_color[current_location[1]][current_location[0]] = 1
        check_color -= 1
    else:
        matrix_color[current_location[1]][current_location[0]] = 0
        check_color += 1
    # Удаление старого положения в матрице
    matrix_location[current_location[1]][current_location[0]] = 0
    
    # Обновление текущего положения муравья
    if current_location[0] == 0:
        current_location[0] = int(matrix_size[0]) - 1
    else:
        current_location[0] -= 1
    # Внесение нового положения муравья в матрицу
    matrix_location[current_location[1]][current_location[0]] = 1

    # Изменение направления взгляда муравья
    if matrix_color[current_location[1]][current_location[0]] == 0:
        current_direction = 'down'
    else:
        current_direction = 'up'

Движение вправо:

In [10]:
def go_right():
    global matrix_color, matrix_location, current_location, current_direction, check_color

    if matrix_color[current_location[1]][current_location[0]] == 0:
        matrix_color[current_location[1]][current_location[0]] = 1
        check_color -= 1
    else:
        matrix_color[current_location[1]][current_location[0]] = 0
        check_color += 1

    matrix_location[current_location[1]][current_location[0]] = 0

    if current_location[0] == int(matrix_size[0]) - 1:
        current_location[0] = 0
    else:
        current_location[0] += 1

    matrix_location[current_location[1]][current_location[0]] = 1

    if matrix_color[current_location[1]][current_location[0]] == 0:
        current_direction = 'up'
    else:
        current_direction = 'down'

Движение вверх:

In [11]:
def go_up():
    global matrix_color, matrix_location, current_location, current_direction, check_color

    if matrix_color[current_location[1]][current_location[0]] == 0:
        matrix_color[current_location[1]][current_location[0]] = 1
        check_color -= 1
    else:
        matrix_color[current_location[1]][current_location[0]] = 0
        check_color += 1

    matrix_location[current_location[1]][current_location[0]] = 0

    if current_location[1] == 0:
        current_location[1] = int(matrix_size[1]) - 1
    else:
        current_location[1] -= 1

    matrix_location[current_location[1]][current_location[0]] = 1

    if matrix_color[current_location[1]][current_location[0]] == 0:
        current_direction = 'left'
    else:
        current_direction = 'right'

Движение вниз:

In [12]:
def go_down():
    global matrix_color, matrix_location, current_location, current_direction, check_color

    if matrix_color[current_location[1]][current_location[0]] == 0:
        matrix_color[current_location[1]][current_location[0]] = 1
        check_color -= 1
    else:
        matrix_color[current_location[1]][current_location[0]] = 0
        check_color += 1

    matrix_location[current_location[1]][current_location[0]] = 0

    if current_location[1] == int(matrix_size[1]) - 1:
        current_location[1] = 0
    else:
        current_location[1] += 1

    matrix_location[current_location[1]][current_location[0]] = 1

    if matrix_color[current_location[1]][current_location[0]] == 0:
        current_direction = 'right'
    else:
        current_direction = 'left'

Сформируем итоговую функцию, отвещающую за перемещение муравья по торической решетке

In [13]:
def go():
    global current_direction, matrix_color, matrix_location, current_location, current_direction, check_color
    if current_direction == 'left':
        go_left()
    elif current_direction == 'right':
        go_right()
    elif current_direction == 'up':
        go_up()
    elif current_direction == 'down':
        go_down()

Создадим функцию сохранения промежуточных результатов

In [14]:
def create_fig(location, direction, colors):
    global iter_counter, save_directory
    fig = plt.figure()
    plt.scatter(location[0], location[1], marker=directions.get(direction), c='r', s=800)
    fig.patch.set_facecolor('white')
    plt.imshow(colors, cmap=cm.gray, alpha=1, vmin=0, vmax=1)
    plt.xticks([])
    plt.yticks([])
    fig.savefig(f"{save_directory}/{iter_counter}.png")

Создадим необходимые счетчики

In [15]:
# Количество итераций до выполнения условия
iter_counter = 0 

# Количество раз, когда поле становилось белым
total_white_counter = 0

# Номер итерации, на которой поле становилось белым
total_white_iter_counter = []

Эволюцию ячеек плоского тора выполним в виде цикла while, имитирующего цикл с постусловием

In [16]:
# Первая итерация проводится вне цикла
if iter_counter == 0:
    # create_fig(current_location, current_direction, matrix_color)
    iter_counter += 1
    go()
    # create_fig(current_location, current_direction, matrix_color)
    
# Вторая и последующие итерации в цикле    
if iter_counter == 1:
    while True:
        iter_counter += 1
        go()
        # create_fig(current_location, current_direction, matrix_color)
        
        # Проверка матрицы цвета на полностью белое состояние
        if check_color == 0:
            total_white_counter += 1
            total_white_iter_counter.append(iter_counter)
            
        # Условия выхода из цикла:
        # 1. Поле снова исходного цвета (белое);
        # 2. Положение муравья равно начальному   
        if check_color == 0 and list(start_location) == list(current_location):
            break

Вывод результатов:

In [17]:
print('Всего итераций до возвращения в исходное положение: ', iter_counter)
print('В ходе выполнения поле становилось белым (раз): ', total_white_counter)
print('Поле становилось белым на итерациях: ', total_white_iter_counter)

Всего итераций до возвращения в исходное положение:  96
В ходе выполнения поле становилось белым (раз):  1
Поле становилось белым на итерациях:  [96]


Блок, позволяющий создать анимацию из промежуточных отчетов

In [18]:
# gif_name = '4x4.gif'
# fps = 3
# file_list = []
# for i in range(iter_counter + 1):
    # file_list.append(glob.glob(f'{save_directory}\\{i}.png')[0])
# clip = mpy.ImageSequenceClip(file_list, fps=fps)
# clip.write_gif('{}'.format(gif_name), fps=fps)

На рисунке представлены результаты расчетов для торической решетки 2х2:

![SegmentLocal](gifs\2x2.gif "segment")

Аналогично для решетки 3х3:

![SegmentLocal](gifs\3x3.gif "segment")

И для торической решетки 4х4:

![SegmentLocal](gifs\4x4.gif "segment")

В таблице представлены результаты расчетов для разных вариантов размера решетки

In [19]:
results = pd.read_excel('results.xlsx')
results

,Сторона тора (n),"Количество раз, когда поле становилось белым (m)","Количество ходов, через которое поле первый раз становилось белым (l)",Количество ходов до возвращения муравья в исходное положение (k)
0,2,1,8,8
1,3,3,22,66
2,4,1,96,96
3,5,5,2342,11710
4,6,1,4592,4592
5,7,7,9166514,64165598
6,8,1,11502464,11502464


## Выводы

Из полученных данных видно, что у торов с нечётным n число m равно числу n, а у торов с чётным n число m равно единице.

Возникает гипотеза, что так дело будет обстоять всегда, для произвольных n.

Из этой гипотезы возникает теорема: если n=p, где р – простое нечётное число, причём первый раз, когда поле становилось, муравей не был в исходной клетке, но смотрел в ту же сторону, что и вначале, то m=n. 